# API

In [50]:
from flask import request
from flask import Flask
import urllib.parse
import requests
from urllib.parse import urlencode
import datetime
import hashlib
import socket
import qrcode
import json

app = Flask(__name__)

## Minimal Example

Run the code below and open http://localhost:3001/ in your browser to see the message `Hello, World!`. When you are done, click the stop button on the left of the cell below.

In [ ]:
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

app.run(port=3001, host="0.0.0.0")

## Routing Example

The url consists of a few different components. The most relevant parts now are:
- The hostname (ex: "localhost", "google.com" or "192.168.1.1")
- The port (ex: "3001", "80" or "443")
- The path (ex: "/", "joke" "/poems/roses" or "/poems/math").

The "/"" path is usually referred to as the "root" path. The path in the URL allows the API to have different functionality attached to it. Run the code below and open the /poems/roses path: http://localhost:3001/poems/roses in your browser to see a different message. Try editing the URL in your browser to access the two other paths as well. Notice the "hello_world()" function was added to the api as the root path handler earlier. Try to access it by changing the path to "/". Is it still available?

In [ ]:
@app.route('/poems/roses')
def roses_poem_path_handler():
    return 'Roses are red, violets are blue, I am a poet, and I know it!'

@app.route('/poems/math')
def math_poem_path_handler():
    return 'Math is fun, it is true, I love math, and so should you!'

@app.route('/joke')
def joke_path_handler():
    return 'Why did the chicken cross the road? To get to the other side!'

app.run(port=3001, host="0.0.0.0")

## URL Parameter Example

Url's can have parameters in them. The Query String in the URL takes the following shape: `?key1=value1&key2=value2`. As an example, run the server and open http://localhost:3001/calculator?a=1&b=2&c=3 in your browser. Try to modify the values of a, b and c.

In [ ]:
@app.route('/calculator')
def calculator_path_handler():
    a = int(request.args.get('a'))
    b = int(request.args.get('b'))
    c = int(request.args.get('c'))
    result = a + b + c
    return f'{a} + {b} + {c} = {result}'

app.run(port=3001, host="0.0.0.0")

Some characters will break the url. For example, a url can not have spaces: http://localhost:3001/people?name=Peter Parker. The space between Peter and Parker will break the url. To fix this we use URL encoding, which replaces invalid characters with a % and a hexadecimal number. Spaces will for example be replaced with a + or %20: http://localhost:3001/people?name=Peter%20Parker. String URL Parameters should always be URL encoded by the client and the server should always decode them. Run the code below and open the link with the URL encoded version of Peter Parker.

In [ ]:
@app.route('/people')
def people_path_handler():
    # Flask automatically decodes the name
    name = request.args.get('name')
    
    return f"Decoded: {name}"

app.run(port=3001, host="0.0.0.0")

Below is an example of how to encode and decode a string programmatically.

In [ ]:
text = "Some kind of #weird @strange /message with + symbols."
encoded = urllib.parse.quote(text)
decoded = urllib.parse.unquote(encoded)

print(f"Original: {text}")
print(f"Encoded: {encoded}")
print(f"Decoded: {decoded}")

## URL Path Parameter Example

Some times it makes sense to make a part of the URL a paraemeter. Run the code below and open http://localhost:3001/people/Peter%20Parker/hobbies and http://localhost:3001/people/Batman/hobbies. Notice how the first name is URL encoded when you open the URL in your browser, and the server automatically decodes it.

In [ ]:
@app.route('/people/<name>/hobbies')
def people_hobbies_path_handler(name):
    
    hobby_list = {
        "Peter Parker": ["Photography", "Science", "Web-slinging"],
        "Batman": ["Money", "Gadgets", "Justice"],
    }
    
    hobbies = hobby_list.get(name)
    
    # Respond with 404 page not found http status code if the person does not exist
    if hobbies is None:
        return "Person not found", 404
    
    return hobbies

app.run(port=3001, host="0.0.0.0")

# JSON Example

Most API's today use JSON as the data format. This makes it easy to read for computers, and every programming language is able to parse it. Run the code below and open http://localhost:3001/json/example in your browser to see a JSON response.

In [ ]:
@app.route("/json/example")
def json_example_path_handler():
    return {
        "databaseName": "people",
        "numEntries": 2,
        "readOnly": True,
        "entries": {
            "size": 2,
            "values": [
                { "name": "Alice", "age": 25 },
                { "name": "Bob", "age": 30 }
            ]
        }
    }

app.run(port=3001, host="0.0.0.0")

# HTTP Request Methods

So far we have been using the browser to access the api. The browser uses http GET method. The most common methods are:
- GET: Used to request data from a server.
- POST: Used to send data to a server to create a resource.
- DELETE: Used to delete a resource.
- PUT: Used to update a resource.

A "resource" usually refers to some kind of information stored somewhere like a database.

If you want to send structured information to an API, the POST method is usually used as it is able to send a body with the request. While URL parameters can be used for this, it is not suitable for large amounts of information, and is typically used for search parameters such as filtering by name in a phone number database.

Run the [json_api_demo.py](json_api_demo.py) code to host an endpoint that accepts POST requests. Notice the endpoint is set to POST using `methods=['POST']` next to the route, and run the code below to fetch the data from the API.

In [ ]:
url = 'http://localhost:3003/sum'

payload = {
    'a': 1,
    'b': 2,
    'c': 3
}

response = requests.post(url, json=payload)

print(response.text)

# Parse the response from JSON String to Python Dictionary
data = response.json()

# Extract the sum field
result = data['sum']

print("Sum:", result)

Run the code below to perform the same operation using the GET endpoint on the same path. This can also be done from your browser.

In [ ]:
base_url = 'http://localhost:3003/sum'

url_params = {'a': 1, 'b': 2, 'c': 3}
# Programmatically constructing the Query String
query_string = urlencode(url_params)
url = f"{base_url}?{query_string}"
print("Url:", url)

response = requests.get(url)
print(response.text)
print("Sum:", data['sum'])

## Ports Example

Your computers network card has a large number of "ports". An application, like the webserver/API, can attach itself to one of these ports, and when another application such as a web browser requests information from your computer, it will do so with a specific port/application/endpoint in mind.

Run the first webserver below as before, and then create a new webserver on a different port by running [api_ports_demo.py](api_ports_demo.py). The first endpoint can be accessed on http://localhost:3001, and the second on http://localhost:3002. Test it in your browser and notice the difference in URL's responses. Remeber to close both webservers when you are done. The first is closed by pressing stop as before, and the second is stopped by clicking the terminal below and pressing CTRL+C.

In [ ]:
app.run(port=3001, host="0.0.0.0")

The default port for the http protocol is 80. Try starting the server on port 80 and open the url in your browser without specifying a port http://localhost. NB! If you have an existing application running on port 80, you will get an access denied error. If this is the case, move along to the next step instead of wasting time on debugging it.

In [ ]:
app.run(port=80, host="0.0.0.0")

The default port for https (http secure) is 443. Try to open it https://localhost. You will likely get a warning saying the site is not secure. What makes a https endpoint secure is TLS certificates. These are not covered in this tutorial, but now you know what the security message means. Stop the sever when you are done.

In [ ]:
app.run(port=443, host="0.0.0.0")

## Hostname Example

So far you have accessed the api using "localhost" as your ipaddress/origin/hostname. Hostnames are a human readable shorthand for IP addresses, the same way your name is a shorthand for your phone number.

Other devices on your network will not be able to find your API using "localhost". Many computers however have mDNS (Multicast DNS), which allows other devices on your network to find your computer by its configured name/hostname.

Run the code below to find your hostname, or run the "`hostname`" command in a terminal. This might not work on all devices, and does not work by default on Arduino without an mDNS library.

In [ ]:
# Get the current hostname
hostname = socket.gethostname()
print("hostname:", hostname)
print("url:", f"http://{hostname}:3001")
print("-----------------------------")

app.run(port=3001, host="0.0.0.0")

Now try to use your local IP Address instead of the hostname. You can find your local IP Address by running the code below, or by running "`ipconfig`" in your terminal and looking for the IPv4 Address on your WiFi/Ethernet card depending on which type of connection you are currently using.

In [ ]:
# Get the current IP address
ip_address = socket.gethostbyname(socket.gethostname())
print("ip_address:", ip_address)
print("url:", f"http://{ip_address}:3001")
print("-----------------------------")

app.run(port=3001, host="0.0.0.0")

## Accessing endpoint from a different device

You have so far only accessed the API from the same computer that is hosting it. Try to connect your phone and computer to the same network (WiFi hotsport will also work), and use the IP Address from earlier to access the API from your phone. Note some networks don't allow Peer To Peer connections, and so this might not work on your Institutional WiFi. You can also try to use the hostname. Remember to run the server and include the port, as well as stopping the server when you are done.

NB! Run all the cells down to the app.run() one before attempting to connect.

In [ ]:
hostname = socket.gethostname()

print("hostname:", f"http://{hostname}:3001")
print("url:", f"http://{hostname}:3001")
qrcode.make(f"http://{hostname}:3001")

In [ ]:
ip_address = socket.gethostbyname(hostname)

print("IP Address:", ip_address)
print("url:", f"http://{ip_address}:3001")
qrcode.make(f"http://{ip_address}:3001")

NB! Run all the cells down to the app.run() one before attempting to connect.

In [ ]:
app.run(port=3001, host="0.0.0.0")

## Access control

Many API's will limit by which address they can be accessed to avoid man-in-the-middle attacks. Try to run the server without the `host="0.0.0.0"`. It will now only allow connections by "localhost", and refuse connections by IP Address. Try using both localhost and ip address from your pc, and ip address from your phone. Only the localhost on your pc will work.

NB! Run all the cells down to the app.run() one before attempting to connect.

In [ ]:
ip_address = socket.gethostbyname(hostname)

print("IP Address:", ip_address)
print("Ip url:", f"http://{ip_address}:3001")
print("localhost url:", f"http://localhost:3001")
qrcode.make(f"http://{ip_address}:3001")

In [ ]:
app.run(port=3001)

# Environment Variables

Passwords, API keys, and other sensitive information should never be hardcoded. Similarly, configuration details that vary between development, testing, and production should be stored in environment variables. These variables allow you to pass configuration to your application without altering the code, acting as parameters for the entire codebase. This is very similar to how one would pass parameters to a function. They are stored in the terminal and, when hosting your API on a cloud server, you typically use a secure method to enter them. For example, to configure the port number and host dynamically, use environment variables. Run the following command in your terminal to execute the example [env_demo.py](env_demo.py):

Windows PowerShell
```bash
$env:PORT=3001; $env:HOST="0.0.0.0"; python Demos/api/env_demo.py
```

Mac Terminal
```bash
PORT=3001 HOST="0.0.0.0" python Demos/api/env_demo.py
```

Linux Terminal
```bash
export PORT=3001 HOST="0.0.0.0" python Demos/api/env_demo.py
```

# Dotenv File

During testing and development it is impractical to pass every environment variable as a command line argument. Instead, you can use a `.env` file to store these variables. This file would typically include sensitive information such as passwords and API keys, and should never be in your git repository, **including private ones**. Removing the file from your repository after the fact will not get rid of it, because the file will still remain in your git history. For this reason, the .env file should be added to the [.gitignore](.gitignore) file. Typically one would create a [example.env](example.env) file as a template that contains all the variables without the actual values. The developer then makes a copy of it named [.env](.env). Copy the example file, name it .env and enter the values for port and host. Run the code below to see how the .env file is loaded and used.

The file should be in the same folder as this notebook, and named `".env"`. Its contents when you are done should be:
```
PORT=3001
HOST=0.0.0.0
```

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the PORT_NUMBER variable
port_number = os.getenv("PORT")
host = os.getenv("HOST")

# Check if the PORT_NUMBER environment variable is set, otherwise throw an error
if not port_number:
    raise ValueError("PORT_NUMBER environment variable is not set")

# Check if the HOST environment variable is set, otherwise throw an error
if not host:
    raise ValueError("HOST environment variable is not set")

print("The value of the PORT env variable is: ", port_number)
print("The value of the HOST env variable is: ", host)

# Passwords

Passwords should never be stored as plain text. Instead they should be hashed. A hashing algorithm will take a password and use it to generate a semi-random string of text that can only be reproduced using that exact password. This means that even if the database is stolen, the hacker can not use the passwords to log in. Typically one would use a common hashing algorithm such as sha256, however since a hashing algorithm by design is semi-random and always produces the same output for the same input, this means someone could potentially generate a large database of the 10 million most common passwords, and then simply compare the hashes. To prevent this it is necessary to "salt" the password, which typically means simply adding a random string to it. Run the code below to see an example.

In [ ]:
alice_password = "12345678"
arne_password = "a"

salt = "this will be added to every password before hashing, and if changed the passwords will all become invalid"

user_database = {
    "alice": {
        "username": "alice",
        "email": "alice@example.com",
        "password_hash": "bc4b5f48b605e8d6ce1eaf06cedac6b34d0519a6ed65bcf3f6706627d4354beb"
    },
    "arne": {
        "username": "arne",
        "email": "arne@example.com",
        "password_hash": "addcf84a5578884c8735594791e62045ea353864a79c1a934da9e96bdf517178"
    }
}

def hash_password(password, salt):
    # Concatenate the password and salt
    salted_password = salt + password
    
    # Hash the salted password using SHA256 algorithm
    return hashlib.sha256(salted_password.encode()).hexdigest()

def check_password(username, password, salt):
    # Get the user from the database
    user = user_database.get(username)
    
    # If the user does not exist, return False
    if user is None:
        return False
    
    # Hash the password
    hashed_password = hash_password(password, salt)
    
    stored_password_hash = user["password_hash"]
    
    # Check if the hashed password matches the one stored in the database
    password_is_correct = hashed_password == stored_password_hash
    
    return password_is_correct

print("Alice password:", alice_password)
print("Alice hash:", hash_password(alice_password, salt))
print("Check Alice's password: ", check_password("alice", alice_password, salt))

print("\nArne password:", arne_password)
print("Arne hash:", hash_password(arne_password, salt))
print("Check Arne's password: ", check_password("arne", arne_password, salt))

print("\nUsing different salt:")
different_salt = "since this salt was not used when hashing the passwords in the database, checks using this salt will fail"
print("Check Alice's password: ", check_password("alice", alice_password, different_salt))
print("Check Arne's password: ", check_password("arne", arne_password, different_salt))


## Advanced: Access Tokens and Full Demo

If you just want to learn the basics, stop here. If you are interested in a full demonstration of how to design an API with authentication, keep reading.

Some times an API should only allow people do access their own information. For example in spotify, you should be the only one able to change your account information. To identify a user, it would be necessary to add the password to every http request. This is not very secure considering many use the same passwords on multiple websites, and if someone were to intercept one of the requests they would have access to all the users accounts. Instead, an access token is used. A token is a random string of text that is generated by the server when you log in, and is then stored in the web browser and sent with every request. Tokens will typically expire after a certain amount of time, and to refresh them the user must log back in. The server can then check if the token is valid, and if it is, it can be used to identify the user. The token is typically sent in the header of the request, and not in the URL Query String or the Body.

The file [access_token_demo.py](access_token_demo.py) contains an example of a complete API. It serves an API using [access_token_demo_database.json](access_token_demo_database.json) as a database, and gets its environment variables from [access_token_demo.env](access_token_demo.env). Run the demo script to continue.


### Example API Docs

The API above has the following paths/endpoints:

- /users
  - [POST]: Takes a username, password, age and email, and creates a new user.
- /users/\<username>
  - [DELETE]: Deletes the user with the given username.
- /users/\<username>/profile
  - [GET]: Returns the profile of the user with the given username.
  - [PUT]: Updates the user with the given username.
- /users/\<username>/grades
  - [GET]: Returns the grades of the user with the given username.
  - [POST]: Creates a document of the users grades.
  - [DELETE]: Deletes all the grades of the user with the given username.
- /users/\<username>/tokens
  - [POST]: Creates a new access token based on a username, password and device_name.
  - [GET]: Lists all access tokens related to this account.
- /users/\<username>/tokens/\<token>
  - [GET] Returns the information about the token.
  - [DELETE]: Deletes the token (same as logging out).


Run the following cells to show how the client should interact with the API 

### Delete Example Users

In [51]:
print(requests.delete("http://localhost:3001/database").text)

{"message":"Database cleared"}



### Creating a new user

In [52]:
# Send a POST request to create a new user
response = requests.post("http://localhost:3001/users", json={
    "username": "Arne",
    "password": "password123",
    "age": 29,
    "email": "arne@example.com",
    "device_name": "Arne's PC",
    "roles": ["teacher", "student"]
})

print("Http Status Code: ", response.status_code, response.reason)

body = response.json()
print("Http Response Body: ")
print(json.dumps(body, indent=4))

arne_access_token = body["access_token"]["token"]

Http Status Code:  200 OK
Http Response Body: 
{
    "access_token": {
        "device_name": "Arne's PC",
        "expires": 1732223960.094133,
        "token": "1aHQCEKpiAmlt-GsoR_QtzfKBXvkAqQpvxTwj6fT2S0",
        "username": "Arne"
    },
    "url": "/users/Arne",
    "user": {
        "age": 29,
        "email": "arne@example.com",
        "username": "Arne"
    }
}


### Get user profile using the access token

If one attempts to access an endpoint locked behind authorization without the access token, the request will be refused.

In [53]:
response = requests.get("http://localhost:3001/users/Arne/profile")

print("Http Stats Code:", response.status_code, response.reason)
print(response.text)

Http Stats Code: 401 UNAUTHORIZED
{"error":"Missing access token"}



When adding the access token, the response will be successful. The access token is prefixed with "Bearer " to indicate the type of authorization (this is an industry standard).

In [54]:
auth_header = {
    "Authorization": f"Bearer {arne_access_token}" 
}

response = requests.get("http://localhost:3001/users/Arne/profile", headers=auth_header)

print("Http Stats Code:", response.status_code, response.reason)

body = response.json()
print("Body:")
print(json.dumps(body, indent=4))

Http Stats Code: 200 OK
Body:
{
    "age": 29,
    "email": "arne@example.com",
    "username": "Arne"
}


### Adding Grades

In [55]:
response = requests.post("http://localhost:3001/users/Arne/grades", headers=auth_header, json={
    "math": 5,
    "english": 4
})

response.json()

{'english': 4, 'math': 5}

### Updating Grades

When a resource alredy exists, a POST endpoint should typically fail. POST is for creating new resources, not updating them. Some times however, developers are lazy. The developer of this perticular endpoint added no guard against this, and so the grades are simply quietly updated.

In [56]:
requests.post("http://localhost:3001/users/Arne/grades", headers=auth_header, json={
    "math": 3,
    "english": 2
}).json()

{'english': 2, 'math': 3}

### Updateing User Profile

In the case of the user, the POST request will fail if the username is taken. Imagine if someone tries to make a new user, and instead updates an existing one because they picked the same name! The code below was intended to update the users age to 30, but it fails because in reality a user data update should use the UPDATE endpoint. You can tell which one should be used by reading the [documentation](#example-api-docs) or [source code](access_token_demo.py) of the API.

In [57]:
response = requests.post("http://localhost:3001/users", json={
    "username": "Arne",
    "password": "password123",
    "age": 30,
    "email": "arne@example.com",
    "device_name": "Arne's PC",
    "roles": ["teacher", "student"]
})

print("Http Status Code: ", response.status_code, response.reason)
print("Http Response Body: ", response.json())

Http Status Code:  400 BAD REQUEST
Http Response Body:  {'error': 'User already exists'}


Now lets try doing it the right way (or at least the way the current backend developer intended). Notice how the POST endpoint for creating a new user includes additional information, that does not make sense to include in the PUT endpoint for updating a user.

In [58]:
response = requests.put(
    "http://localhost:3001/users/Arne/profile",
    headers=auth_header,
    json={
        "username": "Arne",
        "age": 30,
        "email": "arne@example.com",
    }
)

print("Http Status Code: ", response.status_code, response.reason)
print("Http Response Body: ", response.json())

Http Status Code:  200 OK
Http Response Body:  {'age': 30, 'email': 'arne@example.com', 'username': 'Arne'}


### Log Out

Lets log out by invalidating/deleting the access token.

In [59]:
response = requests.delete(
    f"http://localhost:3001/users/Arne/tokens/{arne_access_token}",
    headers=auth_header,
)

print("Http Status Code: ", response.status_code, response.reason)
print("Http Response Body: ", response.json())

Http Status Code:  200 OK
Http Response Body:  {'message': 'Access token deleted'}


### Using Deleted Access Token

Requests using the old token will now be rejected. This is how you are able to remotely log out devices from your social media accounts, and the reason why a device name is provided when logging in.

In [60]:
requests.get("http://localhost:3001/users/Arne/profile", headers=auth_header).json()

{'error': 'Invalid access token'}

### Log In

Logging in means creating a new access token.

In [61]:
response = requests.post("http://localhost:3001/users/Arne/tokens", json={
    "username": "Arne",
    "password": "password123",
    "device_name": "Arne's PC"
})

arnes_pc_access_token = response.json()["token"]

print(json.dumps(response.json(), indent=4))

{
    "device_name": "Arne's PC",
    "expires": 1732224015.876241,
    "token": "4U1R2DyNEiYfhPY7A6c2xumotJfAEusBO1VCx03I1pU",
    "username": "Arne"
}


Lets also make another access token for Arne's phone.

In [62]:
response = requests.post("http://localhost:3001/users/Arne/tokens", json={
    "username": "Arne",
    "password": "password123",
    "device_name": "Arne's Phone"
})

arnes_phone_access_token = response.json()["token"]

print(json.dumps(response.json(), indent=4))

{
    "device_name": "Arne's Phone",
    "expires": 1732224019.741825,
    "token": "PHceT5tOpALI71PwlyjlbYmJH1d9zEXiB3mkETnOY9U",
    "username": "Arne"
}


And his work computer.

In [63]:
arnes_work_pc_access_token = requests.post(
    "http://localhost:3001/users/Arne/tokens",
    json={
        "username": "Arne",
        "password": "password123",
        "device_name": "Arne's Work PC"
    }
).json()["token"]

Using the new tokens, we should be able to fetch the profile of the user.

In [64]:
arnes_pc_auth_header = { "Authorization": f"Bearer {arnes_pc_access_token}"}
arnes_phone_auth_header = { "Authorization": f"Bearer {arnes_phone_access_token}" }
arnes_work_pc_auth_header = { "Authorization": f"Bearer {arnes_work_pc_access_token}"}

In [65]:
requests.get("http://localhost:3001/users/Arne/profile", headers=arnes_pc_auth_header).json()

{'age': 30, 'email': 'arne@example.com', 'username': 'Arne'}

In [66]:
requests.get("http://localhost:3001/users/Arne/profile", headers=arnes_phone_auth_header).json()

{'age': 30, 'email': 'arne@example.com', 'username': 'Arne'}

In [67]:
requests.get("http://localhost:3001/users/Arne/profile", headers=arnes_work_pc_auth_header).json()

{'age': 30, 'email': 'arne@example.com', 'username': 'Arne'}

### List Access Tokens

The API has an endpoint for listing the access tokens of a user. This can for example be used to list what devices are currently logged into the account. List endpoints typically have filters in the URL Parameters (Query String). In this case there is a "limit" parameter that can be used to limit the number of tokens returned. For endpoints with thousands of entries, this is necessary to avoid crashing the client/server/database as well as keeping the user experience fast and responsive.

In [72]:
response = requests.get(
    "http://localhost:3001/users/Arne/tokens?limit=2",
    headers=arnes_pc_auth_header
)

tokens = response.json()

print("Tokens:")
print(json.dumps(tokens, indent=4))

Tokens:
{
    "count": 2,
    "limit": 2,
    "tokens": [
        {
            "device_name": "Arne's PC",
            "expires": 1732224015.876241,
            "token": "4U1R2DyNEiYfhPY7A6c2xumotJfAEusBO1VCx03I1pU",
            "username": "Arne"
        },
        {
            "device_name": "Arne's Phone",
            "expires": 1732224019.741825,
            "token": "PHceT5tOpALI71PwlyjlbYmJH1d9zEXiB3mkETnOY9U",
            "username": "Arne"
        }
    ],
    "total_count": 3
}


Now lets fetch all of them and see what devices are logged in. Notice the endpoint is configured with a default limit of 10.

In [77]:
tokens = requests.get(
    "http://localhost:3001/users/Arne/tokens",
    headers=arnes_pc_auth_header
).json()["tokens"]


print("Connected Devices:")
for token in tokens:
    print(token["device_name"])

Connected Devices:
Arne's PC
Arne's Phone
Arne's Work PC


Lets log out the 2 other devices.

In [78]:
for token in tokens:
    if token["device_name"] == "Arne's PC":
        continue
    
    response = requests.delete(
        f"http://localhost:3001/users/Arne/tokens/{token['token']}",
        headers=arnes_pc_auth_header
    )
    
    if response.status_code == 200:
        print(f"Device {token['device_name']} was removed")

Device Arne's Phone was removed
Device Arne's Work PC was removed


Now lets list the tokens again

In [81]:
response = requests.get(
    "http://localhost:3001/users/Arne/tokens",
    headers=arnes_pc_auth_header
)

print(json.dumps(response.json(), indent=4))

{
    "count": 1,
    "limit": 10,
    "tokens": [
        {
            "device_name": "Arne's PC",
            "expires": 1732224015.876241,
            "token": "4U1R2DyNEiYfhPY7A6c2xumotJfAEusBO1VCx03I1pU",
            "username": "Arne"
        }
    ],
    "total_count": 1
}


### Get access token information

The token endpoint also has a GET method that can be used to get information about a specific token, such as the device and when it expires (forces user to log in again).

In [86]:
print(json.dumps(requests.get(
    f"http://localhost:3001/users/Arne/tokens/{arnes_pc_access_token}",
    headers=arnes_pc_auth_header,
).json(), indent=4))

{
    "device_name": "Arne's PC",
    "expires": 1732224015.876241,
    "token": "4U1R2DyNEiYfhPY7A6c2xumotJfAEusBO1VCx03I1pU",
    "username": "Arne"
}


### View the database

After all these operations, there is information stored in the database. In this case the database is simply the [access_token_demo_database.json](access_token_demo_database.json) file. Open it to have a look at how the (API)[access_token_demo.py] stores its information behind the scenes. When you are done, run the cell below to clear the database.

In [88]:
print(requests.delete("http://localhost:3001/database").text)

{"message":"Database cleared"}

